### Section I:Data download, transform and cleaning from Wiki web

In [3]:
pip install beautifulsoup4 requests

     |████████████████████████████████| 122kB 5.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### First, get a local copy of the Wikipedia article:

In [380]:
import urllib.request
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req=urllib.request.urlopen(url)
article=req.read().decode()

with open('List_of_postal_codes_of_Canada.html', 'w') as fo:
    fo.write(article)

Second, load it and parse it with Beautiful Soup. Extract all the table tags and search for the one with headings corresponding to the data we want. Iterate over the rows and pulling out the columns we want. Then write the cell text a new file. 

In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
#Load article, turn into soup and get the <table>s
article=open('List_of_postal_codes_of_Canada.html').read()
soup=BeautifulSoup(article, 'html.parser')
tables=soup.find_all('table', class_='sortable')
#tables

In [6]:
# Search through the tables for the one with the headings we want.
for table in tables:
    ths=table.find_all('th')    #th is for bold heading columns, td for columns, tr is for rows.
    headings=[th.text.strip() for th in ths]
    if headings[:3]==['Postal Code', 'Borough', 'Neighborhood']:
        break
        
headings


['Postal Code', 'Borough', 'Neighborhood']

In [7]:
#Extract the columns we want and write to a semicolon-delimited text file.
with open('List_of_postal_codes_of_Canada.txt', 'w') as fo:
    for tr in table.find_all('tr'):     #td is for columns, tr is for rows.
        tds=tr.find_all('td')
        if not tds:
            continue
        Postal_Code, Borough, Neighborhood = [td.text.strip() for td in tds[:3]]
        print(';'.join([Postal_Code, Borough, Neighborhood]), file=fo)
        #print(tr)
#fo=open('List_of_postal_codes_of_Canada.txt', 'r')
#print(fo.read())    

Read and Clean the txt file with Pandas.

In [8]:
import pandas as pd
df=pd.read_table('List_of_postal_codes_of_Canada.txt', delimiter=';', names=headings)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df.shape

(180, 3)

In [10]:
#drop rows with "not assigned" Borough.
df=df[df['Borough']!='Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df.shape

(103, 3)

In [12]:
df.index

Int64Index([  2,   3,   4,   5,   6,   8,   9,  11,  12,  13,
            ...
            151, 152, 153, 156, 157, 160, 165, 168, 169, 178],
           dtype='int64', length=103)

In [13]:
#Assign column Neighborhood with 'Not assigned' with the same name as column Borough.
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood']= ['Borough']
df=df.rename(columns={'Postal Code': 'PostalCode'})
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
df.shape

(103, 3)

### Section II: Use Geocoder to get the coordinates of the Toronto neighborhood.


In [15]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         393 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

In [16]:
from geopy import geocoders
from geopy.geocoders import Nominatim
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [17]:
pip install geocoder

     |████████████████████████████████| 102kB 11.6MB/s ta 0:00:01
     |████████████████████████████████| 92kB 5.3MB/s eta 0:00:011
     |████████████████████████████████| 829kB 17.1MB/s eta 0:00:01     |████▍                           | 112kB 17.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


Try out **geocoder.arcgis** with an address and example zipcode.

In [19]:
import geocoder 
g = geocoder.arcgis('M5G')
print(g.latlng)
print(type(g.latlng))

[43.65607218800005, -79.38565318999997]
<class 'list'>


In [20]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format('M5A'))
    lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
                              
print(lat_lng_coords)
print(lat_lng_coords[0], lat_lng_coords[1])
print(latitude)
print(longitude)

[43.65096410900003, -79.35304116399999]
43.65096410900003 -79.35304116399999
43.65096410900003
-79.35304116399999


In [29]:
df['PostalCode']

2      M3A
3      M4A
4      M5A
5      M6A
6      M7A
      ... 
160    M8X
165    M4Y
168    M7Y
169    M8Y
178    M8Z
Name: PostalCode, Length: 103, dtype: object

In [21]:
import numpy as np
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords =[]
#lat_lng_coords.index=np.linspace(range(len(df.index)))

# loop until you get the coordinates
for pcode in df['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(pcode))
    lat_lng_coords.append(g.latlng)
    #lat_lng_coords=
    
#print(lat_lng_coords)

In [22]:
#Convert lat_lng_coords to a dataframe.
pcode_df=pd.DataFrame(lat_lng_coords)
pcode_df.columns=['Latitude', 'Longitude']
pcode_df.head()

,Latitude,Longitude
0,43.752935,-79.335641
1,43.728102,-79.311890
2,43.650964,-79.353041
3,43.723265,-79.451211
4,43.661790,-79.389390


In [23]:
df=df.reset_index(drop=True) #reset index so that can combine df and pcode_df.
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
df.index
pcode_df.index

RangeIndex(start=0, stop=103, step=1)

Combine neighborhood dataframe from Wikipedia and the dataframe containing coordinates info.

In [25]:
import pandas as pd
df_canada=pd.concat([df, pcode_df], axis=1)
df_canada.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [26]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
len(df_canada['Borough'].unique()),
df_canada.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


### Create a new dataframe containing only Toronto neighborhoods.

In [27]:
#df_canada['Borough'].str.rsplit(' ', 1)
#df_canada['Borough'].str.split(None, 1)
df_TO=df_canada[df_canada.Borough.str.endswith('Toronto')]
print(df_TO.shape)
df_TO.reset_index(inplace=True, drop=True)
print(df_TO['Borough'])
df_TO.head()

(39, 5)
0     Downtown Toronto
1     Downtown Toronto
2     Downtown Toronto
3     Downtown Toronto
4         East Toronto
5     Downtown Toronto
6     Downtown Toronto
7     Downtown Toronto
8     Downtown Toronto
9         West Toronto
10    Downtown Toronto
11        West Toronto
12        East Toronto
13    Downtown Toronto
14        West Toronto
15        East Toronto
16    Downtown Toronto
17        East Toronto
18     Central Toronto
19     Central Toronto
20     Central Toronto
21     Central Toronto
22        West Toronto
23     Central Toronto
24     Central Toronto
25        West Toronto
26     Central Toronto
27    Downtown Toronto
28        West Toronto
29     Central Toronto
30    Downtown Toronto
31     Central Toronto
32    Downtown Toronto
33    Downtown Toronto
34    Downtown Toronto
35    Downtown Toronto
36    Downtown Toronto
37    Downtown Toronto
38        East Toronto
Name: Borough, dtype: object


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
len(df_TO['Borough'].unique()),
df_TO.shape[0]))

The dataframe has 4 boroughs and 39 neighborhoods.


### Explore and cluster the neighborhoods in Toronto

In [29]:
import json  # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans  # import k-means from clustering stage
import folium  # map rendering library

print('Libraries imported.')

Libraries imported.


### Use geopy library to get the latitude and longitude values of Toronto. 

In order to define an instance of the geocoder, we need to define a user_agent. The agent name for our case is <em>TO_explorer</em>.

In [30]:
address= 'Toronto, Ontario, Canada'

geolocator=Nominatim(user_agent='TO_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [31]:
# create map of Toronto using latitude and longitude values
map_toronto=folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, borough, neighborhood in zip(df_TO['Latitude'], df_TO['Longitude'], df_TO['Borough'], df_TO['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#bcbcbc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursqure Credentials and Version

In [32]:
CLIENT_ID = '0PRPCB4BHWLH333U4O3PNSSK4ZCOMA3HQFJDNSJNAM35OUC0' # your Foursquare ID
CLIENT_SECRET = '4FAYAOZY4WD3NVYIMRJRHSO1PJOZOWXXVND1OZA0WJSBXQLI' # your Foursquare Secret
VERSION = '2020622' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0PRPCB4BHWLH333U4O3PNSSK4ZCOMA3HQFJDNSJNAM35OUC0
CLIENT_SECRET:4FAYAOZY4WD3NVYIMRJRHSO1PJOZOWXXVND1OZA0WJSBXQLI


Let's explore the first neighborhood in our dataframe.  
Get the neighborhood's name.

In [408]:
df_TO.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


In [33]:
df_TO.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.

In [34]:
neighborhood_latitude = df_TO.loc[0, 'Latitude']
neighborhood_longitude = df_TO.loc[0, 'Longitude']
neighborhood_name=df_TO.loc[0, 'Neighborhood']

print("Latitude and longitude values of {} are {}, {}.".format(neighborhood_name,
                                                          neighborhood_latitude,
                                                          neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65096410900003, -79.35304116399999.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.  
First, let's create the GET request URL.

In [35]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id=0PRPCB4BHWLH333U4O3PNSSK4ZCOMA3HQFJDNSJNAM35OUC0&client_secret=4FAYAOZY4WD3NVYIMRJRHSO1PJOZOWXXVND1OZA0WJSBXQLI&ll=43.65096410900003,-79.35304116399999&v=20200622&radius=500&limit=100'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url
results=requests.get(url).json()
results
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '57e0745a498ea809dbf75f68',
   'name': 'Souk Tabule',
   'location': {'address': '494 Front St E',
    'crossStreet': 'at Bayview',
    'lat': 43.65375556880743,
    'lng': -79.35439006096168,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.65375556880743,
      'lng': -79.35439006096168}],
    'distance': 329,
    'cc': 'CA',
    'neighborhood': 'Canary District',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['494 Front St E (at Bayview)',
     'Toronto ON',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d1c0941735',
     'name': 'Mediterranean Restaurant',
     'pluralName': 'Mediterranean Restaurants',
     'shortName': 'Mediterranean',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
      'suffix': '.png'},
     'p

In [36]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
venues=results['response']['groups'][0]['items']

nearby_venues=json_normalize(venues) #flatten JSON
print(nearby_venues.shape)
nearby_venues.head(2)

#filter columns
filtered_columns=['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues=nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns
nearby_venues.columns=[col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues.head()

(27, 22)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
2,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843
3,BATLgrounds,Athletics & Sports,43.647088,-79.351306
4,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127


In [39]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

27 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

Create a function to repeat the same process to all the neighborhoods in Manhattan

In [410]:
print(df_TO.shape)
df_TO.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


In [40]:
CLIENT_ID = '0PRPCB4BHWLH333U4O3PNSSK4ZCOMA3HQFJDNSJNAM35OUC0' # your Foursquare ID
CLIENT_SECRET = '4FAYAOZY4WD3NVYIMRJRHSO1PJOZOWXXVND1OZA0WJSBXQLI' # your Foursquare Secret
VERSION = '20200622' # Foursquare API version
LIMIT = 100
#ACCESS_TOKEN="21NGLY0W2TFEHBYGQC11TV2SDJWUSGZIGWMT3DG0KUUNXLT4"
#ACCESS_TOKEN='Y0YMA4F5LONOSGRQ1PFK2F0JEL0ZCXDNMUQLAJMH4FIPDBV5'

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
       
        # make the GET request
        #print(url) 
        results = requests.get(url).json()["response"]['groups'][0]['items']
       
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
          
    return(nearby_venues)


#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [323]:
df_TO.tail(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4Y,Downtown Toronto,Church and Wellesley,43.666659,-79.381472
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.648700,-79.385450


In [41]:
df_TO['Neighborhood'].unique().shape

(39,)

In [42]:
toronto_venues=getNearbyVenues(names=df_TO['Neighborhood'],
                              latitudes=df_TO['Latitude'],
                              longitudes=df_TO['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [43]:
#check the size of the reulting dataframe.
print(toronto_venues.shape)
toronto_venues.head()

(1597, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop


In [44]:
#Let's check how many venues were returned for each neighborhood.
toronto_venues.groupby('Neighborhood').count().shape

(39, 6)

In [45]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 unique categories.


### Analyze Each Neighborhood

In [46]:
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
toronto_onehot.drop('Neighborhood', axis=1, inplace=True)
print(toronto_onehot.shape)
toronto_onehot.head(2)

(1597, 227)


,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [47]:
toronto_venues.head(2)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue


In [48]:
toronto_venues['Neighborhood'].shape

(1597,)

In [49]:
type(toronto_venues['Neighborhood'].values)

numpy.ndarray

In [50]:
toronto_onehot.shape
#toronto_onehot['Neighborhood']

(1597, 227)

In [51]:
# one hot encoding
#toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
print(toronto_onehot.columns)
#add neighborhood column back to dataframe
neighborhood=toronto_venues['Neighborhood'].values
toronto_onehot.insert(0, 'Neighborhood', neighborhood, allow_duplicates=False)
#print(toronto_onehot.columns)

fixed_columns=list(toronto_onehot.columns)
               
#print(fixed_columns)
toronto_onehot=toronto_onehot[fixed_columns]

toronto_onehot.head()

Index(['Accessories Store', 'Afghan Restaurant', 'American Restaurant',
       'Antique Shop', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop',
       ...
       'Trail', 'Train Station', 'Vegetarian / Vegan Restaurant',
       'Veterinarian', 'Video Game Store', 'Vietnamese Restaurant', 'Wine Bar',
       'Wine Shop', 'Women's Store', 'Yoga Studio'],
      dtype='object', length=227)


,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Let's examine the new dataframe size.

In [52]:
toronto_onehot.shape

(1597, 228)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurence of each category

In [53]:
toronto_onehot['Neighborhood'].unique().shape

(39,)

In [54]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.015625,0.031250,0.000000,0.000000,0.015625,0.000000,0.031250,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.015625,0.046875,0.109375,0.000000,0.000000,0.000000,0.00,0.000000,0.015625,0.000000,0.015625,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.000000,0.015625,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.00,0.000000,0.000000,0.015625,0.000000,0

In [55]:
# Confirm the new size
toronto_grouped.shape

(39, 228)

#### Let's print each neighborhood along with the top 5 most common venues

In [56]:
num_top_venues=5
for hood in toronto_grouped['Neighborhood']:
    print('----'+hood+'----')
    temp=toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns=['venue', 'freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.11
1    Cocktail Bar  0.05
2     Cheese Shop  0.03
3          Bakery  0.03
4  Breakfast Spot  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.07
2                   Diner  0.04
3              Restaurant  0.04
4  Thrift / Vintage Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.03
4           Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.06
1                Café  0.06
2                Park  0.05
3   French Restaurant  0.05
4  Italian Restaurant  0.03


----Central Bay Street----
      

### Put this info into a *pandas* dataframe   

First, write a function to sort the venues in descending order.

In [57]:
toronto_grouped.iloc[1, :][1:].sort_values(ascending=False).index.values[0:5]

array(['Coffee Shop', 'Café', 'Diner', 'Pizza Place', 'Gift Shop'],
      dtype=object)

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [59]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

#create columns according to number of top venues
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#create a new dataframe
neighborhoods_venues_sorted=pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for n in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[n, 1:]=return_most_common_venues(toronto_grouped.iloc[n, :], num_top_venues)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()
    

(39, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Breakfast Spot,Seafood Restaurant,Hotel,Cheese Shop,Café,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Diner,Pizza Place,Gift Shop,Restaurant,Thrift / Vintage Store,Accessories Store,Caribbean Restaurant,Boutique
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Salon / Barbershop,Steakhouse,Italian Restaurant,Sushi Restaurant,Gym
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,French Restaurant,Park,Gym / Fitness Center,Italian Restaurant,Restaurant,Bar,Lounge,Speakeasy
4,Central Bay Street,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Plaza,Japanese Restaurant,Bubble Tea Shop,Clothing Store,Italian Restaurant,Spa,Hotel


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [60]:
toronto_grouped.shape

(39, 228)

In [76]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels=kmeans.labels_
labels
#labels.shape

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [62]:
df_TO.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


In [63]:
df_TO.shape

(39, 5)

In [77]:
# add clustering labels
#neighborhoods1_venues_sorted=neighborhoods_venues_sorted.copy().drop('Cluster Labels', axis=1)
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop('Cluster Labels', axis=1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', labels)

TO_merged = df_TO

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
TO_merged = TO_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

print(TO_merged.shape)
TO_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Café,Trail,Athletics & Sports,Thai Restaurant,Seafood Restaurant,Mexican Restaurant,Food Truck,French Restaurant,Chocolate Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0,Coffee Shop,Café,Sushi Restaurant,Yoga Studio,Distribution Center,Sculpture Garden,Sandwich Place,Pharmacy,Park,Middle Eastern Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Bar,Cosmetics Shop,Hotel,Café,Italian Restaurant,Theater,Movie Theater
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub,Restaurant,American Restaurant,Clothing Store,Italian Restaurant,Gym
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Pub,Trail,Asian Restaurant,Church,Farmers Market,Farm,Fast Food Restaurant,Falafel Restaurant,Dog Run


Finally, visualize the resulting clusters.

In [74]:
TO_merged['Neighborhood']

0                             Regent Park, Harbourfront
1           Queen's Park, Ontario Provincial Government
2                              Garden District, Ryerson
3                                        St. James Town
4                                           The Beaches
5                                           Berczy Park
6                                    Central Bay Street
7                                              Christie
8                              Richmond, Adelaide, King
9                          Dufferin, Dovercourt Village
10    Harbourfront East, Union Station, Toronto Islands
11                             Little Portugal, Trinity
12                         The Danforth West, Riverdale
13             Toronto Dominion Centre, Design Exchange
14         Brockton, Parkdale Village, Exhibition Place
15                       India Bazaar, The Beaches West
16                       Commerce Court, Victoria Hotel
17                                      Studio D

In [68]:
TO_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [78]:
#create map
map_clusters=folium.Map(location=[latitude, longitude], zoom_start=10)

#set color scheme for the clusters
x = np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TO_merged['Latitude'], TO_merged['Longitude'], TO_merged['Neighborhood'], TO_merged['Cluster Labels']):
    #cluster.astype(int)
    label=folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=0.7).add_to(map_clusters)
map_clusters